In [1]:
import pandas as pd

In [2]:
df_enroll = pd.read_csv("data/raw_data/train/enrollment_train.csv")
df_log = pd.read_csv("data/raw_data/train/log_train.csv")
df_truth = pd.read_csv("data/raw_data/train/truth_train.csv")

In [3]:
df_enroll.head()

,enrollment_id,username,course_id
0,772,5981,course-v1:TsinghuaX+70800232X+2015_T2
1,773,1544995,course-v1:TsinghuaX+70800232X+2015_T2
2,774,1072798,course-v1:TsinghuaX+70800232X+2015_T2
3,776,561867,course-v1:TsinghuaX+70800232X+2015_T2
4,777,1368125,course-v1:TsinghuaX+70800232X+2015_T2


In [4]:
df_log.head()

,Unnamed: 0,enrollment_id,source,event,object,time
0,0,772,browser,navigate,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl,2015-09-27T15:42:59
1,1,772,browser,navigate,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl,2015-09-27T15:43:12
2,2,773,browser,video,3dac5590435e43b3a65a9ae7426c16db,2015-10-19T19:37:42
3,3,773,browser,video,3dac5590435e43b3a65a9ae7426c16db,2015-10-19T19:33:27
4,4,773,browser,video,3dac5590435e43b3a65a9ae7426c16db,2015-10-19T19:33:30


In [5]:
df_log['event'].unique()

array(['navigate', 'video', 'page_close', 'discussion', 'access',
       'problem'], dtype=object)

In [6]:
df_truth = pd.read_csv("data/raw_data/train/truth_train.csv")

In [7]:
import pandas as pd
import numpy as np

# Assuming df_log and df_enroll are already loaded

# Create a dictionary to map actions to categories
action_to_category = {
    'navigate': 'about',
    'video': 'video',
    'page_close': 'html',
    'discussion': 'discussion',
    'access': 'course',
    'problem': 'problem'
}

# Create a mapping from enrollment_id to course_id
enrollment_to_course = dict(zip(df_enroll['enrollment_id'], df_enroll['course_id']))

# Add course_id to df_log
df_log['course_id'] = df_log['enrollment_id'].map(enrollment_to_course)

# Create a mapping of (course_id, object) to event
course_object_to_event = df_log.groupby(['course_id', 'object'])['event'].first().to_dict()

# Group df_log by course_id and object, and get the minimum time
grouped = df_log.groupby(['course_id', 'object'])['time'].min().reset_index()

# Create the df_object DataFrame
df_object = pd.DataFrame({
    'course_id': grouped['course_id'],
    'module_id': grouped['object'],
    'category': grouped.apply(lambda row: action_to_category.get(course_object_to_event.get((row['course_id'], row['object']), ''), 'chapter'), axis=1),
    'children': np.nan,
    'start': pd.to_datetime(grouped['time'])
})

# Ensure category is one of the allowed values
allowed_categories = ['about', 'chapter', 'course', 'course_info', 'html', 'outlink', 
                      'problem', 'sequential', 'static_tab', 'vertical', 'video', 
                      'combinedopenended', 'peergrading', 'discussion', 'dictation']
df_object.loc[~df_object['category'].isin(allowed_categories), 'category'] = np.nan

# Sort the DataFrame
df_object = df_object.sort_values(['course_id', 'start'])

# Reset index
df_object = df_object.reset_index(drop=True)

print(df_object.head())

               course_id                                          module_id  \
0  CAU/08112500x/2015_T2                   a7ca8a91072147fea23cbb9309a594f6   
1  CAU/08112500x/2015_T2  i4x-CAU-08112500x-video-05b6e2ea2cfe418b9dbb21...   
2  CAU/08112500x/2015_T2                   3T6XwoiMKgol57cm29Rjy8FXVFcIomxl   
3  CAU/08112500x/2015_T2                   12bef5b2ae004ff791db67a4cf4dcd23   
4  CAU/08112500x/2015_T2                   262880bd6ef04ae589178b3f4a0c18dc   

  category  children               start  
0   course       NaN 2015-09-07 08:55:01  
1    video       NaN 2015-09-07 08:55:03  
2    about       NaN 2015-09-07 08:57:59  
3   course       NaN 2015-09-07 13:11:53  
4   course       NaN 2015-09-07 13:11:53  


In [8]:
df_truth.head()

,0,772,1
0,1,773,1
1,2,774,1
2,3,776,0
3,4,777,1
4,5,779,1


In [9]:
df_date = pd.read_csv("data/raw_data/date.csv")

In [10]:
df_date.head()

,Unnamed: 0,course_id,from,to
0,0,course-v1:NJU+010101+2016_T1,2016-03-28,2016-05-03
1,1,course-v1:TsinghuaX+00510663X+2015_T2,2015-10-12,2015-11-17
2,2,TsinghuaX/10450034_1X/2015_T2,2015-11-02,2015-12-07
3,3,course-v1:TsinghuaX+00612643X+2016_T2,2016-10-10,2016-11-15
4,4,MITx/15_390x_2015_T1/2015_T1,2015-07-03,2015-08-08


In [11]:
df_date['start'] = pd.to_datetime(df_date['from'])
df_date['end'] = pd.to_datetime(df_date['to'])
df_date['duration'] = (df_date['end']-df_date['start']).dt.days/7
df_date.groupby('duration').nunique()

,Unnamed: 0,course_id,from,to,start,end
duration,,,,,,
5.000000,41,41,30,30,30,30
5.142857,203,203,84,84,84,84


In [12]:
df_enroll.head()

,enrollment_id,username,course_id
0,772,5981,course-v1:TsinghuaX+70800232X+2015_T2
1,773,1544995,course-v1:TsinghuaX+70800232X+2015_T2
2,774,1072798,course-v1:TsinghuaX+70800232X+2015_T2
3,776,561867,course-v1:TsinghuaX+70800232X+2015_T2
4,777,1368125,course-v1:TsinghuaX+70800232X+2015_T2


In [48]:
df_log.groupby('event').nunique()

,enrollment_id,time,source,object
event,,,,
access,91370,2151338,2,1674
discussion,40666,600992,1,1
navigate,120499,914766,1,59
page_close,84225,1110587,1,1
problem,41180,791141,2,2133
video,76120,740763,1,2026
wiki,31386,87947,1,1


In [13]:
df_log.sample(n=50)

,Unnamed: 0,enrollment_id,source,event,object,time,course_id
18734755,18734755,315696,browser,navigate,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl,2016-03-09T21:02:01,course-v1:TsinghuaX+30640014X+2016_T1
10196586,10196586,163073,server,access,12b8d104c5c944b9969656c3c042e850,2016-07-09T21:49:16,course-v1:TsinghuaX+JRFX01+2016_TS
18757732,18757732,315997,browser,problem,8f37f191035147dfaf5da367b6ac4aba,2016-03-12T13:21:17,course-v1:TsinghuaX+30640014X+2016_T1
15906596,15906596,238422,browser,video,511f8a039b9f41718fca5c02ff03bdfc,2016-10-15T17:38:57,course-v1:TsinghuaX+20250064+2016_T2
5359113,5359113,88990,browser,video,a39a1c4e77a9436e8db8505e7c328572,2016-07-16T10:10:48,course-v1:TsinghuaX+30640014+2016_T2
28866380,28866380,463795,browser,problem,L1:L1_Problem_A,2015-06-14T10:10:33,course-v1:MITx+6_00_1x+2015_T1
20257620,20257620,344124,browser,navigate,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl,2016-04-04T22:43:48,course-v1:TsinghuaX+80512073X+2016_T1
13485634,13485634,225139,browser,problem,ad8421dbc7364b0491046931f426de53,2016-09-19T17:20:01,course-v1:TsinghuaX+00680082_1X+2016_T2
9043690,9043690,154793,browser,video,68f8a372535941e890231b3686054231,2017-02-21T13:59:29,course-v1:TsinghuaX+10610224X+2017_T1
28208857,28208857,449788,browser,navigate,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl,2016-09-19T21:49:20,course-v1:TsinghuaX+10620204X+2016_T2


In [14]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score


In [15]:
def load_data(data_type='train'):
    """Load the required datasets."""
    base_path = f'data/raw_data/{data_type}/'
    enrollment = pd.read_csv(f'{base_path}enrollment_{data_type}.csv')
    log = pd.read_csv(f'{base_path}log_{data_type}.csv')
    truth = pd.read_csv(f'{base_path}truth_{data_type}.csv', header=None, names=['enrollment_id', 'dropout'])
    return enrollment, log, truth

def load_additional_data():
    """Load date and object data."""
    date_df = pd.read_csv('data/kddcup15/date.csv')
    object_df = pd.read_csv('data/kddcup15/object.csv')
    return date_df, object_df


In [16]:
def preprocess_enrollment(enrollment):
    """Preprocess the enrollment data."""
    # Create copies of username and course_id for encoding
    #enrollment['username_encoded'] = enrollment['username']
    #enrollment['course_id_encoded'] = enrollment['course_id']
    
    #le = LabelEncoder()
    #enrollment['username_encoded'] = le.fit_transform(enrollment['username_encoded'])
    #enrollment['course_id_encoded'] = le.fit_transform(enrollment['course_id_encoded'])
    return enrollment


def process_date_data(date_df, enrollment_df):
    """Process date data and merge with enrollment data."""
    date_df['from'] = pd.to_datetime(date_df['from'])
    date_df['to'] = pd.to_datetime(date_df['to'])
    date_df['course_duration'] = (date_df['to'] - date_df['from']).dt.days

    return pd.merge(enrollment_df, date_df[['course_id', 'course_duration']], on='course_id', how='left')

def process_object_data(object_df, enrollment_df):
    """Process object data and create features."""
    # Count the number of modules per course
    modules_per_course = object_df.groupby('course_id').size().reset_index(name='module_count')
    
    # Count the number of each category per course
    category_counts = object_df.groupby(['course_id', 'category']).size().unstack(fill_value=0)
    category_counts.columns = [f'category_{col}_count' for col in category_counts.columns]
    
    # Merge all object features
    object_features = pd.merge(modules_per_course, category_counts, on='course_id', how='left')
    
    return pd.merge(enrollment_df, object_features, on='course_id', how='left')


In [17]:
def aggregate_log_data(log):
    """Aggregate log data to create features."""
    return log.groupby('enrollment_id').agg({
        'source': 'count',
        'event': lambda x: x.nunique(),
        'object': 'nunique'
    }).rename(columns={
        'source': 'activity_count',
        'event': 'unique_events',
        'object': 'unique_objects'
    })

def merge_datasets(enrollment, log_features, truth=None):
    """Merge the preprocessed datasets."""
    df = pd.merge(enrollment, log_features, on='enrollment_id', how='left')
    df.fillna(0, inplace=True)
    if truth is not None:
        df = pd.merge(df, truth, on='enrollment_id')
    return df

def engineer_features(df):
    """Perform feature engineering."""
    df['event_object_ratio'] = df['unique_events'] / (df['unique_objects'] + 1)
    df['activity_per_event'] = df['activity_count'] / (df['unique_events'] + 1)
    return df

def prepare_features(df):
    """Prepare features for model training or prediction."""
    X = df.drop(['enrollment_id', 'dropout','username','course_id'], axis=1)
    y = df['dropout']
    return X, y


def engineer_additional_features(df):
    """Engineer additional features using the new data."""
    df['modules_per_day'] = df['module_count'] / df['course_duration']
    df['category_diversity'] = df[[col for col in df.columns if col.startswith('category_')]].gt(0).sum(axis=1)
    return df


In [18]:
def train_model(X, y):
    """Train the Random Forest model."""
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X, y)
    return model

def evaluate_model(model, X, y):
    """Evaluate the model and return performance metrics."""
    y_pred = model.predict(X)
    y_pred_proba = model.predict_proba(X)[:, 1]
    accuracy = accuracy_score(y, y_pred)
    auc = roc_auc_score(y, y_pred_proba)
    return accuracy, auc

In [19]:
def calculate_metrics(y_true, y_pred, y_pred_proba):
    """Calculate accuracy, AUC, precision, and recall."""
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_proba)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return accuracy, auc, precision, recall

In [20]:
# train_enrollment, train_log, train_truth = load_data('train')
# test_enrollment, test_log, test_truth = load_data('test')

# train_enrollment = preprocess_enrollment(train_enrollment)
# test_enrollment = preprocess_enrollment(test_enrollment)

# train_log_features = aggregate_log_data(train_log)
# test_log_features = aggregate_log_data(test_log)


#         # Pro

In [21]:
# train_df = merge_datasets(train_enrollment, train_log_features, train_truth)
# test_df = merge_datasets(test_enrollment, test_log_features, test_truth)



In [24]:
# X_test, y_test = prepare_features(test_df)


In [22]:
train_df.head()

,enrollment_id,username,course_id,activity_count,unique_events,unique_objects,dropout
0,772,5981,course-v1:TsinghuaX+70800232X+2015_T2,2,1,1,1
1,773,1544995,course-v1:TsinghuaX+70800232X+2015_T2,21,2,2,1
2,774,1072798,course-v1:TsinghuaX+70800232X+2015_T2,74,3,4,1
3,776,561867,course-v1:TsinghuaX+70800232X+2015_T2,19,3,2,0
4,777,1368125,course-v1:TsinghuaX+70800232X+2015_T2,4,1,1,1


In [36]:

    # Load all data
    train_enrollment, train_log, train_truth = load_data('train')
    test_enrollment, test_log, test_truth = load_data('test')
    #date_df, object_df = load_additional_data()

    # Preprocess enrollment data
    train_enrollment = preprocess_enrollment(train_enrollment)
    test_enrollment = preprocess_enrollment(test_enrollment)

    # Process date and object data
    #train_enrollment = process_date_data(date_df, train_enrollment)
    #train_enrollment = process_object_data(object_df, train_enrollment)
    #test_enrollment = process_date_data(date_df, test_enrollment)
    #test_enrollment = process_object_data(object_df, test_enrollment)

    # Aggregate log data
    train_log_features = aggregate_log_data(train_log)
    test_log_features = aggregate_log_data(test_log)

    # Merge datasets
    train_df = merge_datasets(train_enrollment, train_log_features, train_truth)
    test_df = merge_datasets(test_enrollment, test_log_features, test_truth)

    # Engineer features
    train_df = engineer_features(train_df)
    test_df = engineer_features(test_df)
    #train_df = engineer_additional_features(train_df)
    #test_df = engineer_additional_features(test_df)

    # Prepare features
    X_train, y_train = prepare_features(train_df)
    X_test, y_test = prepare_features(test_df)

    X_test = X_test.drop(columns=['Unnamed: 0'])

    # Train the model
    model = train_model(X_train, y_train)

    # Evaluate on training data
    y_train_pred = model.predict(X_train)
    y_train_pred_proba = model.predict_proba(X_train)[:, 1]
    train_metrics = calculate_metrics(y_train, y_train_pred, y_train_pred_proba)
    
    print("Training Metrics:")
    print(f"Accuracy: {train_metrics[0]:.4f}")
    print(f"AUC: {train_metrics[1]:.4f}")
    print(f"Precision: {train_metrics[2]:.4f}")
    print(f"Recall: {train_metrics[3]:.4f}")

    # Evaluate on test data
    y_test_pred = model.predict(X_test)
    y_test_pred_proba = model.predict_proba(X_test)[:, 1]
    test_metrics = calculate_metrics(y_test, y_test_pred, y_test_pred_proba)
    
    print("\nTest Metrics:")
    print(f"Accuracy: {test_metrics[0]:.4f}")
    print(f"AUC: {test_metrics[1]:.4f}")
    print(f"Precision: {test_metrics[2]:.4f}")
    print(f"Recall: {test_metrics[3]:.4f}")



Training Metrics:
Accuracy: 0.8904
AUC: 0.8883
Precision: 0.8940
Recall: 0.9707

Test Metrics:
Accuracy: 0.7995
AUC: 0.7504
Precision: 0.8419
Recall: 0.9058
